# Drowsiness Detection

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [3]:
# the threshold defines the number of frames which
# determines whether the driver is drowsy or not
DROWSY_THRESHOLD = 15

In [5]:
# cascade classifiers taken from OpenCV
face_classifier = cv2.CascadeClassifier('../model/haarcascade_frontalface_default.xml')
eyes_classifier = cv2.CascadeClassifier('../model/haarcascade_eye.xml')

model=load_model('../model/cnn_model.h5')

# convert the image to rgb scale
def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

close_frame = 0
prev_eyes = []

# find the eyes from the face, and then feed it into our trained CNN model to detect drowsiness
def face_ROI(frame):
    
    global close_frame, prev_eyes
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(frame_gray, 1.3, 3, minSize=(30, 30))
    
    for (x,y,w,h) in faces:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        face_gray = frame_gray[y:y+h, x:x+w]
        face = frame[y:y+h, x:x+w]
        eyes = eyes_classifier.detectMultiScale(face_gray)
    
        new_eyes = []
        for (x2,y2,w2,h2) in eyes:
            
            # only detect features on th upper half of the face
            if y2+h2/2 > h/2: 
                continue
            
            new_eyes.append((x2,y2,w2,h2))

        if new_eyes:
            prev_eyes = new_eyes
        else:
            close_frame += 1
            
        for (x2,y2,w2,h2) in prev_eyes:
            cv2.rectangle(face,(x2,y2),(x2+w2,y2+h2),(0,0,255),2)
            eye = face_gray[y2:y2+h2, x2:x2+w2]
            eye = cv2.cvtColor(eye, cv2.COLOR_GRAY2BGR)
            try:
                eye = cv2.resize(eye, (50, 50))
            except Exception as e:
                break
            eye = eye/255
            eye = np.array(eye)
            eye = np.expand_dims(eye, axis=0)
            prediction = model.predict(eye)
            if(prediction[0][0] < 0.5):
                close_frame += 1
            else:
                close_frame = max(0, close_frame-1)
        
        # alert the driver if they exceed drowsiness threshold
        if close_frame >= DROWSY_THRESHOLD:
            cv2.putText(frame, 
                        "DROWSY!!!", 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (0, 0, 255), 
                        2, 
                        cv2.LINE_4)    

    cv2.imshow('TEAM12', frame)

## Start Video Capture and Drowsiness Classification

In [6]:
vid = cv2.VideoCapture(0)
print(vid.get(cv2.CAP_PROP_FPS))
while(True):
      
    # Capture the video frame by frame
    ret, frame = vid.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame,(500,250))
    face_ROI(frame)
    
    # press the 'q' button to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

5.0
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
